In [1]:
# imports
import numpy as np
np.random.seed(42)
from tensorflow.keras.utils import set_random_seed
set_random_seed(42)
import pandas as pd
import seaborn as sns
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# loading dataset
df = sns.load_dataset('iris')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
# converting the categorical target feature into a numerical one
df['species'] = df['species'].map({'setosa':0, 'versicolor':1, 'virginica':2})

In [4]:
# creating X & y; converting a vector of integers (y) into a one-hot-encoded vector
X = df.drop(['species'], axis=1)
y = to_categorical(y=df['species'], num_classes=3)

In [5]:
# splitting the data into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
# scaling the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# defining an MLP architecture
model = Sequential()
model.add(Dense(8, activation='relu', input_dim=4))
model.add(BatchNormalization())
model.add(Dense(4, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))

In [8]:
# model compilation
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
# model training
history = model.fit(X_train, y_train, batch_size=50, epochs=500, validation_data=(X_test, y_test), verbose=0)

In [10]:
# model evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print('Loss:', round((score[0] * 100), 2))
print('Accuracy:', round((score[1] * 100), 2))

Loss: 3.79
Accuracy: 100.0


## confusion matrix

In [11]:
# plotting confusion matrix
print(confusion_matrix(
    y_true = np.argmax(y_test, axis=1),
    y_pred = np.argmax(model.predict(X_test, verbose=0), axis=1)
    ))

[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]


## classification report

In [12]:
# getting the argmax of true values
y_test = np.argmax(y_test, axis=1)
print(y_test)
print(y_test.shape)

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
(30,)


In [13]:
# making predictions
y_pred = model.predict(X_test, verbose=0)
print(y_pred)
print(y_pred.shape)

[[1.46292220e-03 7.47362554e-01 2.51174539e-01]
 [9.95336354e-01 3.80969234e-03 8.53979087e-04]
 [1.55674388e-06 2.88254432e-05 9.99969602e-01]
 [6.61468250e-04 9.95102167e-01 4.23638942e-03]
 [6.71521062e-03 9.64803100e-01 2.84817312e-02]
 [9.95017469e-01 4.07857914e-03 9.03929467e-04]
 [4.38200004e-05 9.99942899e-01 1.32893711e-05]
 [2.18495773e-03 2.95701018e-03 9.94858027e-01]
 [2.15295167e-03 9.84773874e-01 1.30731473e-02]
 [3.10490606e-04 9.99518871e-01 1.70679225e-04]
 [2.42167688e-03 4.10330184e-02 9.56545353e-01]
 [9.96932626e-01 2.47188914e-03 5.95434161e-04]
 [9.97937202e-01 1.63995172e-03 4.22901270e-04]
 [9.96326745e-01 2.97788065e-03 6.95448951e-04]
 [9.98677671e-01 1.03430974e-03 2.87929375e-04]
 [7.77796493e-04 9.96596038e-01 2.62610381e-03]
 [3.75873897e-05 5.39241126e-04 9.99423146e-01]
 [6.51720329e-04 9.98877704e-01 4.70608880e-04]
 [7.27662467e-04 9.91641760e-01 7.63058057e-03]
 [1.51957502e-05 2.25089272e-04 9.99759734e-01]
 [9.96530235e-01 2.80763023e-03 6.621395

In [14]:
# getting the argmax of predictions
rounded_pred = np.argmax(y_pred, axis=-1)
print(rounded_pred)
print(rounded_pred.shape)

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
(30,)


In [15]:
# plotting the classification report
print(classification_report(y_test, rounded_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



## easy way of plotting a classification report
    
    print(classification_report(
      y_true = np.argmax(y_test, axis=1), 
      y_pred = np.argmax(model.predict(X_test, verbose=0), axis=1)))